# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [2]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer

from nltk import pos_tag
from collections import defaultdict

In [33]:
#somme global variables
prefixes = ['ante','anti','auto','co','com', 'con','contra', 'contro','de','dis','en','ex','extra','hetero','homo','hyper',
'il', 'im', 'in', 'ir','inter','intra','macro','non','omni','post','pre','pro','sub','sym', 'syn','tele','trans','tri','un','uni','up']

prefix_l2 = ['contra', 'contro','extra','hetero','inter','hyper','intra','macro','trans']

Noun_suffixes = ['acy','al','ance','ence','dom','er','ee','or','ism','ist','ity', 'ty','ment','ness','ship','sion','tion']
Verb_Suffixes  = ['ate','en','ify', 'fy','ize', 'ise']
Adjective_Suffixes = ['able', 'ible','al','ary','ed','ing', 'esque','ful','ic', 'ical','ish','ive','less','y']
Adverb_Suffixes = ['ly', 'ward', 'wise']
plural_suffixes = ['es', 's']
plural_sf = {'ies':'y' , 'ves':'f'}

suffixes = Noun_suffixes+Verb_Suffixes+Adjective_Suffixes+Adverb_Suffixes+plural_suffixes
Notusing_suffixes = ['ious', 'ous']

In [35]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.master_flag = 1
        self.current_dictionary = []
        self.wrong_guesses = []
        #self.strat3_flag = 1
        
    def guess(self, word): # word input example: "_ p p _ e "
        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".") 
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        nodup_dictionary = []
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
                nodup_dictionary.append("".join(set(dict_word)))
                
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(nodup_dictionary)    
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break

        return guess_letter
    
    def guess_strat2(self, word, isstat2=True, m=0):
        clean_word = word[::2].replace("_",".")
        if isstat2:
            clean_word_temp = clean_word
            clean_word = clean_word_temp.strip('.')
            m = len(clean_word_temp) - len(clean_word)
            clean_word = '(.*)'+clean_word+'(.*)'
        # find length of passed word
        len_word = len(clean_word)
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        nodup_dictionary = []
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) > m+1+ len_word or len(dict_word) < len_word:
                continue
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
                nodup_dictionary.append("".join(set(dict_word)))   
                
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary        
        # count occurrence of all characters in possible word matches        
        full_dict_string = "".join(nodup_dictionary)
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common() 
        
        guess_letter = '!'
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break  
        
        return guess_letter
        
    def word_clean_st3(self, word):
        clean_word = word[::2].replace("_",".")
        history_len = len(clean_word)
        # prefix cleaning code   
        pref = ""
        for pf in prefixes:
            if clean_word[:len(pf)] == pf and len(pref) < len(pf):  
                pref = pf
        '''for pf in prefix_l2:
            if re.match(clean_word[:len(pf)], pf) and len(clean_word[:len(pf)].replace(".", "")) => 3:
                pref = pf'''
        if pref != "":
            clean_word = clean_word[len(pref):]

        #final depluralization, as it can only occur in the last place
        suff = ""
        for sf in plural_sf:
            if clean_word[-len(sf):] == sf and len(suff) < len(sf):
                suff = sf
        if suff != "":
            clean_word = clean_word[:-len(suff)] + plural_sf[suff]
        #suffix cleaning code 
        suff = "./"
        while len(suff)>0:
            suff = ""
            for sf in suffixes:
                if clean_word[-len(sf):] == sf and len(suff) < len(sf):
                    suff = sf
            if suff != "":
                clean_word = clean_word[:-len(suff)]
        #final word
        word = " ".join(clean_word).replace(".","_") + ' '
        return word, history_len- len(clean_word)
    
    def guess_strat4(self, word):
        clean_word = word[::2].replace("_",".")
        guess_letter = '!'
        if clean_word.count('.') <= 2 and clean_word[-1] == '.':
            if clean_word[-2] in ('l', 'c', 'h') and 'y' not in self.guessed_letters:
                guess_letter = 'y'
            elif clean_word[-2] == 'e' and 'd' not in self.guessed_letters:
                guess_letter = 'd'
            elif 's' not in self.guessed_letters:
                guess_letter = 's'
            if guess_letter != '!':
                print("Strat4 magic happened!")
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break    
        return guess_letter
    
    def delete_irrelevent_words(self, letter):
        current_dictionary = self.current_dictionary
        #del_list = [word for word in current_dictionary if letter in word]
                
        final_dictionary = [wd for wd in current_dictionary if letter not in wd]
        return final_dictionary
    
    def build_dictionary(self,dictionary_file_location):
        print(" Building dictionary, will take 0.41 sec/1000 words")
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()

        lemmatizer = WordNetLemmatizer()
        snowball = SnowballStemmer(language='english')
        tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

        fuller_dictionary = []
        for word in full_dictionary:
            tk, tg = pos_tag([word])[0]
            lem =lemmatizer.lemmatize(tk, tag_dict.get(tg[0], wordnet.NOUN))
            stm = snowball.stem(word)

            if stm != word and word != lem:
                lem = stm
            sing = word
            for sf in plural_sf:
                if word[-len(sf):] == sf:
                    sing = word[:-len(sf)] + plural_sf[sf]
    
            fuller_dictionary += list(set([word,stm,lem, sing]))
        print(" Removing Duplicates")
        fuller_dictionary = list(dict.fromkeys(fuller_dictionary))
        return fuller_dictionary
    
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.master_flag = 1
        self.wrong_guesses = []   
        #self.strat3_flag = 1
        
        response = self.request("/new_game", {"practice":practice})

        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                if api.master_flag ==1:
                    guess_letter = api.guess(word)
                    if guess_letter == '!':
                        print("Strat 1 returned no solution, changing to strat 2.")
                        api.master_flag = 2
                        api.current_dictionary = api.full_dictionary

                if api.master_flag ==2:
                    guess_letter = api.guess_strat2(word,True)
                    if guess_letter == '!':
                        print("Strat 2 returned no solution, changing to strat 3.")
                        api.master_flag = 3
                        api.current_dictionary = api.full_dictionary

                if api.master_flag ==3:
                    new_word, m = api.word_clean_st3(word)
                    #print("new_word in strat3 reduction:", new_word)
                    if new_word != word:
                        guess_letter = api.guess_strat2(new_word, False,m)
                        if  guess_letter == '!':
                            print("Strat 3 returned no solution, changing to strat 4.")
                            api.master_flag = 4
                            api.current_dictionary = api.full_dictionary
                    else:
                        print("Strat 3 not useful now, changing to strat 4.")
                        api.master_flag = 4
                        api.current_dictionary = api.full_dictionary

                if api.master_flag ==4:
                    new_word, m = word, 0
                    if word.count('_')/len(word) < 0.2:
                        new_word, m = api.word_clean_st3(word)
                    guess_letter = '!'
                    if new_word != word:
                        print("new_word in strat4 reduction:", new_word)
                        guess_letter = api.guess_strat2(new_word, False,m)
                    if guess_letter == '!':
                        guess_letter = api.guess_strat4(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Server response: {0}".format(res))
                
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return (True, self.master_flag)
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return (False, self.master_flag)
                elif status=="ongoing":
                    if res.get('word') == word and self.master_flag == 1: # that is our guess is wrong
                        self.wrong_guesses.append(guess_letter)
                        self.current_dictionary = self.delete_irrelevent_words(guess_letter)
                    if len(self.current_dictionary) == 0 and api.master_flag < 4:
                        self.master_flag += 1
                        print("Current Scope is empty, changing to strat{0}".format(self.master_flag))
                        self.current_dictionary = self.full_dictionary
                    # uppdating the word
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return (status=="success", 0)
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2                                                                                        
        for it in range(num_retry):                                                                                         
            try:                                                                                                            
                response = self.session.request(                                                                            
                    method or "GET",                                                                                        
                    HANGMAN_URL + path,                                                                                     
                    timeout=self.timeout,                                                                                   
                    params=args,                                                                                            
                    data=post_args                                                                                          
                )                                                                                                           
                break                                                                                                       
            except requests.HTTPError as e:                                                                                 
                response = json.loads(e.read())                                                                             
                raise HangmanAPIError(response)                                                                             
            except requests.exceptions.SSLError as e:                                                                       
                if it + 1 == num_retry:                                                                                     
                    raise                                                                                                   
                time.sleep(time_sleep)  

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [36]:
api = HangmanAPI(access_token="16d04ed2733fd37b75934590e1186f", timeout=2000)


 Building dictionary, will take 0.41 sec/1000 words
 Removing Duplicates


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [28]:
practice_counter= 0
practice_success = [[0,0,0,0,0], [0,0,0,0,0]] #fisrt is failure array and second is sucess array
for i in range(40):
    print("Playing game no: ", i)
    (res, desc) = api.start_game(practice=1,verbose=True)
    practice_counter += res
    practice_success[res][desc] += 1
#time.sleep(0.2)
'''[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
'''

#print("Total wins:" , counter)

Playing game no:  0
Successfully start a new game! Game ID: 9904b24a10dd. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: i
Server response: {'game_id': '9904b24a10dd', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ i _ _ _ _ _ _ _ _ _ '}
Guessing letter: n
Server response: {'game_id': '9904b24a10dd', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ n _ _ i _ _ _ n _ _ _ _ _ '}
Guessing letter: o
Server response: {'game_id': '9904b24a10dd', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ o n _ _ i o _ _ n _ _ _ _ _ '}
Strat 1 returned no solution, changing to strat 2.
Guessing letter: c
Server response: {'game_id': '9904b24a10dd', 'status': 'ongoing', 'tries_remains': 6, 'word': 'c o n _ c i o _ _ n _ _ _ _ _ '}
Guessing letter: s
Server response: {'game_id': '9904b24a10dd', 'status': 'ongoing', 'tries_remains': 6, 'word': 'c o n s c i o _ s n _ s s _ s '}
Strat 2 returned no solution, changing to strat 3.
Strat 3 returned no soluti

Successfully start a new game! Game ID: e9d9d37547a1. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'e9d9d37547a1', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ e _ _ e _ _ '}
Guessing letter: r
Server response: {'game_id': 'e9d9d37547a1', 'status': 'ongoing', 'tries_remains': 6, 'word': 'r _ _ _ e _ _ e _ _ '}
Guessing letter: s
Server response: {'game_id': 'e9d9d37547a1', 'status': 'ongoing', 'tries_remains': 6, 'word': 'r _ _ _ e _ _ e s s '}
Guessing letter: l
Server response: {'game_id': 'e9d9d37547a1', 'status': 'ongoing', 'tries_remains': 5, 'word': 'r _ _ _ e _ _ e s s '}
Guessing letter: n
Server response: {'game_id': 'e9d9d37547a1', 'status': 'ongoing', 'tries_remains': 5, 'word': 'r _ _ _ e _ n e s s '}
Guessing letter: t
Server response: {'game_id': 'e9d9d37547a1', 'status': 'ongoing', 'tries_remains': 4, 'word': 'r _ _ _ e _ n e s s '}
Guessing letter: d
Server response: {'game_id': 'e9d9d37547a1', 'status

Successfully start a new game! Game ID: 32545f9130e3. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '32545f9130e3', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ e '}
Guessing letter: i
Server response: {'game_id': '32545f9130e3', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ i _ _ _ _ _ _ e '}
Guessing letter: t
Server response: {'game_id': '32545f9130e3', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ i _ _ _ _ _ _ e '}
Guessing letter: n
Server response: {'game_id': '32545f9130e3', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n i _ _ _ _ _ _ e '}
Guessing letter: s
Server response: {'game_id': '32545f9130e3', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ n i _ _ _ _ _ _ e '}
Guessing letter: c
Server response: {'game_id': '32545f9130e3', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ n i _ _ _ c _ _ e '}
Strat 1 returned no solution, changing to strat 2.
Guessing letter: r
S

Server response: {'game_id': 'c6dc0f31e2e3', 'status': 'ongoing', 'tries_remains': 5, 'word': 's e _ i _ i _ t a t o r i a _ _ _ '}
Guessing letter: l
Server response: {'game_id': 'c6dc0f31e2e3', 'status': 'ongoing', 'tries_remains': 5, 'word': 's e _ i _ i _ t a t o r i a l l _ '}
Guessing letter: c
Server response: {'game_id': 'c6dc0f31e2e3', 'status': 'ongoing', 'tries_remains': 5, 'word': 's e _ i _ i c t a t o r i a l l _ '}
Guessing letter: u
Server response: {'game_id': 'c6dc0f31e2e3', 'status': 'ongoing', 'tries_remains': 4, 'word': 's e _ i _ i c t a t o r i a l l _ '}
Guessing letter: d
Server response: {'game_id': 'c6dc0f31e2e3', 'status': 'ongoing', 'tries_remains': 4, 'word': 's e _ i d i c t a t o r i a l l _ '}
Strat4 magic happened
Guessing letter: y
Server response: {'game_id': 'c6dc0f31e2e3', 'status': 'ongoing', 'tries_remains': 4, 'word': 's e _ i d i c t a t o r i a l l y '}
new_word in strat4 reduction: s e _ i d i c t a t o r i 
Guessing letter: m
Server response

Server response: {'game_id': '3ef816fc2f9a', 'status': 'ongoing', 'tries_remains': 2, 'word': 'a n e n c e p _ a l o t r o p _ i a '}
Guessing letter: h
Server response: {'game_id': '3ef816fc2f9a', 'status': 'success', 'tries_remains': 2, 'word': 'a n e n c e p h a l o t r o p h i a '}
Successfully finished game: 3ef816fc2f9a
Playing game no:  18
Successfully start a new game! Game ID: 06019166806a. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '06019166806a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Server response: {'game_id': '06019166806a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ i _ _ i _ '}
Guessing letter: t
Server response: {'game_id': '06019166806a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ i _ _ i _ '}
Guessing letter: a
Server response: {'game_id': '06019166806a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a _ i _ _ i _ '}
Guessi

...

Successfully start a new game! Game ID: 65d608a67ed1. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: i
Server response: {'game_id': '65d608a67ed1', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ _ i _ _ _ '}
Guessing letter: a
Server response: {'game_id': '65d608a67ed1', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ a _ i _ a _ '}
Guessing letter: c
Server response: {'game_id': '65d608a67ed1', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ a _ i c a _ '}
Guessing letter: l
Server response: {'game_id': '65d608a67ed1', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ l a _ i c a l '}
Guessing letter: t
Server response: {'game_id': '65d608a67ed1', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ l a t i c a l '}
Guessing letter: r
Server response: {'game_id': '65d608a67ed1', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ r _ r _ l a t i c a l '}
S

Successfully start a new game! Game ID: 89fb1a591339. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '89fb1a591339', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Server response: {'game_id': '89fb1a591339', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ i _ _ _ _ _ _ _ _ '}
Guessing letter: a
Server response: {'game_id': '89fb1a591339', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ _ i _ _ a _ _ _ _ a '}
Guessing letter: c
Server response: {'game_id': '89fb1a591339', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ _ i _ _ a _ c _ _ a '}
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
Guessing letter: o
Server response: {'game_id': '89fb1a591339', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ _ i o _ a _ c o _ a '}
Guessing letter: r
Server response: {

Server response: {'game_id': '391cf66f68fb', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i n _ _ _ _ _ n _ _ '}
Guessing letter: c
Server response: {'game_id': '391cf66f68fb', 'status': 'ongoing', 'tries_remains': 4, 'word': 'i n _ _ _ _ _ n _ _ '}
Guessing letter: a
Server response: {'game_id': '391cf66f68fb', 'status': 'ongoing', 'tries_remains': 4, 'word': 'i n _ _ _ _ a n _ _ '}
Guessing letter: t
Server response: {'game_id': '391cf66f68fb', 'status': 'ongoing', 'tries_remains': 4, 'word': 'i n _ _ _ _ a n t _ '}
Guessing letter: s
Server response: {'game_id': '391cf66f68fb', 'status': 'ongoing', 'tries_remains': 4, 'word': 'i n _ _ _ _ a n t s '}
Guessing letter: r
Server response: {'game_id': '391cf66f68fb', 'status': 'ongoing', 'tries_remains': 4, 'word': 'i n _ _ r _ a n t s '}
Guessing letter: v
Server response: {'game_id': '391cf66f68fb', 'status': 'ongoing', 'tries_remains': 3, 'word': 'i n _ _ r _ a n t s '}
Current Scope is empty, changing to strat2
Strat 2 returned 

Server response: {'game_id': 'd1fec96b7287', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ a n _ _ _ '}
Guessing letter: o
Server response: {'game_id': 'd1fec96b7287', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ a n _ _ o '}
Guessing letter: h
Server response: {'game_id': 'd1fec96b7287', 'status': 'ongoing', 'tries_remains': 2, 'word': 'h _ a n _ _ o '}
Guessing letter: g
Server response: {'game_id': 'd1fec96b7287', 'status': 'ongoing', 'tries_remains': 1, 'word': 'h _ a n _ _ o '}
Guessing letter: c
Server response: {'game_id': 'd1fec96b7287', 'status': 'ongoing', 'tries_remains': 1, 'word': 'h _ a n _ c o '}
Guessing letter: u
Server response: {'game_id': 'd1fec96b7287', 'status': 'success', 'tries_remains': 1, 'word': 'h u a n u c o '}
Successfully finished game: d1fec96b7287


"[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)\npractice_success_rate = total_practice_successes / total_practice_runs\nprint('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))\n"

In [29]:
print(practice_success)
print(practice_counter)

[[0, 5, 4, 1, 12], [0, 4, 1, 6, 7]]
18


In [13]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


run 911 practice games out of an allotted 100,000. practice success rate so far = 0.276


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [27]:
for i in range(100):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=True)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Current Scope is empty, changing to strat2
Playing  1  th game
Current Scope is empty, changing to strat2
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: _ n o c c _ e t t i 
Playing  2  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: l a t i _ u n _ i u _ 
new_word in strat4 reduction: l a t i _ u n d i u _ 
Playing  3  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: c e p h a l _ c h _ r _ a t a 
new_word in strat4 reduction: c e p h a l o c h o r _ a t a 
new_word in strat4 reduction: c e p h a l o c h o r _ a t a 
new_word in strat4 reduction: c e p h a l o c h o r _ a t a 
new_word in strat4 reduction: c e p h a l o

Playing  33  th game
Current Scope is empty, changing to strat2
Playing  34  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: _ i _ e r n i _ i _ e 
new_word in strat4 reduction: _ i _ e r n i _ i _ e 
new_word in strat4 reduction: _ i _ e r n i _ 
new_word in strat4 reduction: h i _ e r n i _ 
new_word in strat4 reduction: h i b e r n i _ 
Playing  35  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: e p i p o _ i _ i _ 
new_word in strat4 reduction: e p i p o _ i _ i _ 
new_word in strat4 reduction: e p i p o _ i _ i _ 
new_word in strat4 reduction: e p i p o _ i t i _ 
new_word in strat4 reduction: e p i p o _ i t i _ 
new_word in strat4 reduction: e p i p o _ i t i _ 
new_word in strat4 reduction: e p i p o _ i t 


new_word in strat4 reduction: r n _ a l l 
Playing  69  th game
Current Scope is empty, changing to strat2
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: h a p _ o _ o n t _ 
new_word in strat4 reduction: h a p l o _ o n t _ 
new_word in strat4 reduction: h a p l o _ o n t _ 
Playing  70  th game
Current Scope is empty, changing to strat2
Strat 2 returned no solution, changing to strat 3.
Playing  71  th game
Playing  72  th game
Playing  73  th game
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Strat 3 not useful now, changing to strat 4.
new_word in strat4 reduction: _ e _ i s s e 
new_word in strat4 reduction: _ e _ i s s e 
new_word in strat4 reduction: _ e _ i s s e 
new_word in strat4 reduction: _ e t i s s e 
Playing  74  th game
Playing  75  th game
Playing  76  th game
Current Scope is empty, changing to strat2
Strat 2 returned no solution, cha

In [49]:
counter= 0
success = [[0,0,0,0,0], [0,0,0,0,0]] #fisrt is failure array and second is sucess array
for i in range(100):
    print('Playing ', i, ' th game')
    (res, desc) = api.start_game(practice=0,verbose=True)
    counter += res
    success[res][desc] += 1
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.4)

Playing  0  th game
Successfully start a new game! Game ID: 2951ec21b080. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '2951ec21b080', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e _ _ _ _ '}
Guessing letter: r
Server response: {'game_id': '2951ec21b080', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e r _ _ _ '}
Guessing letter: i
Server response: {'game_id': '2951ec21b080', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ i _ i _ e r _ _ _ '}
Guessing letter: s
Server response: {'game_id': '2951ec21b080', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ i _ i _ e r _ _ s '}
Guessing letter: o
Server response: {'game_id': '2951ec21b080', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ i _ i _ e r o _ s '}
Guessing letter: u
Server response: {'game_id': '2951ec21b080', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ i _ i _ e r o u s '}
Guessing letter: f
Serv

Server response: {'game_id': '3964aec9a4ba', 'status': 'ongoing', 'tries_remains': 4, 'word': 'd e l i r _ u s '}
new_word in strat4 reduction: l i r _ u 
Guessing letter: o
Server response: {'game_id': '3964aec9a4ba', 'status': 'success', 'tries_remains': 4, 'word': 'd e l i r o u s '}
Successfully finished game: 3964aec9a4ba
Playing  4  th game
Successfully start a new game! Game ID: 7cb63146070d. # of tries remaining: 6. Word: _ _ _ _ _ .
Guessing letter: a
Server response: {'game_id': '7cb63146070d', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ '}
Guessing letter: e
Server response: {'game_id': '7cb63146070d', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ e '}
Guessing letter: o
Server response: {'game_id': '7cb63146070d', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ e '}
Guessing letter: i
Server response: {'game_id': '7cb63146070d', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ e '}
Guessing letter: u
Server response: {'game_id'

Server response: {'game_id': '8cfdd30bc7e1', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ e s _ e r _ o r n i n g '}
Guessing letter: t
Server response: {'game_id': '8cfdd30bc7e1', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ e s t e r _ o r n i n g '}
Guessing letter: y
Server response: {'game_id': '8cfdd30bc7e1', 'status': 'ongoing', 'tries_remains': 3, 'word': 'y e s t e r _ o r n i n g '}
Guessing letter: m
Server response: {'game_id': '8cfdd30bc7e1', 'status': 'success', 'tries_remains': 3, 'word': 'y e s t e r m o r n i n g '}
Successfully finished game: 8cfdd30bc7e1
Playing  9  th game
Successfully start a new game! Game ID: 5e1f85392678. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '5e1f85392678', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e _ _ _ _ '}
Guessing letter: r
Server response: {'game_id': '5e1f85392678', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e r _ 

Server response: {'game_id': '6a5f3e314b2b', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ a _ t e r _ _ _ t a '}
Guessing letter: o
Server response: {'game_id': '6a5f3e314b2b', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ a _ t e r _ _ o t a '}
Guessing letter: n
Server response: {'game_id': '6a5f3e314b2b', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a n a _ t e r _ _ o t a '}
Guessing letter: s
Server response: {'game_id': '6a5f3e314b2b', 'status': 'ongoing', 'tries_remains': 4, 'word': 'a n a _ t e r _ _ o t a '}
Guessing letter: l
Server response: {'game_id': '6a5f3e314b2b', 'status': 'ongoing', 'tries_remains': 3, 'word': 'a n a _ t e r _ _ o t a '}
Guessing letter: c
Server response: {'game_id': '6a5f3e314b2b', 'status': 'ongoing', 'tries_remains': 2, 'word': 'a n a _ t e r _ _ o t a '}
Guessing letter: u
Server response: {'game_id': '6a5f3e314b2b', 'status': 'ongoing', 'tries_remains': 1, 'word': 'a n a _ t e r _ _ o t a '}
Guessing letter: d
Server respon

Server response: {'game_id': '4aca39dc9211', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ o _ s i _ e d '}
Guessing letter: g
Server response: {'game_id': '4aca39dc9211', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ o _ s i _ e d '}
Guessing letter: c
Server response: {'game_id': '4aca39dc9211', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ o _ s i _ e d '}
Guessing letter: t
Server response: {'game_id': '4aca39dc9211', 'status': 'ongoing', 'tries_remains': 1, 'word': 't o _ s i _ e d '}
Guessing letter: p
Server response: {'game_id': '4aca39dc9211', 'status': 'failed', 'tries_remains': 0, 'word': 't o _ s i _ e d '}
Failed game: 4aca39dc9211. Because of: # of tries exceeded!
Playing  18  th game
Successfully start a new game! Game ID: f905f5867de3. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'f905f5867de3', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e _ '}
Guessing letter: r
Server response

...

Server response: {'game_id': '243fdac41ad1', 'status': 'ongoing', 'tries_remains': 3, 'word': 'c a l _ _ _ o '}
Guessing letter: n
Server response: {'game_id': '243fdac41ad1', 'status': 'ongoing', 'tries_remains': 2, 'word': 'c a l _ _ _ o '}
Guessing letter: t
Server response: {'game_id': '243fdac41ad1', 'status': 'ongoing', 'tries_remains': 1, 'word': 'c a l _ _ _ o '}
Guessing letter: s
Server response: {'game_id': '243fdac41ad1', 'status': 'ongoing', 'tries_remains': 1, 'word': 'c a l _ _ s o '}
Guessing letter: p
Server response: {'game_id': '243fdac41ad1', 'status': 'ongoing', 'tries_remains': 1, 'word': 'c a l _ p s o '}
Guessing letter: y
Server response: {'game_id': '243fdac41ad1', 'status': 'success', 'tries_remains': 1, 'word': 'c a l y p s o '}
Successfully finished game: 243fdac41ad1
Playing  84  th game
Successfully start a new game! Game ID: 6ac1e7537378. # of tries remaining: 6. Word: _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '6ac1e7537378', 'status'

Server response: {'game_id': '053696146242', 'status': 'failed', 'tries_remains': 0, 'word': '_ a r _ e a u '}
Failed game: 053696146242. Because of: # of tries exceeded!
Playing  88  th game
Successfully start a new game! Game ID: 39e3f404b4f5. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '39e3f404b4f5', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ e '}
Guessing letter: i
Server response: {'game_id': '39e3f404b4f5', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ i _ e '}
Guessing letter: t
Server response: {'game_id': '39e3f404b4f5', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ t _ _ _ i _ e '}
Guessing letter: n
Server response: {'game_id': '39e3f404b4f5', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ t _ _ _ i _ e '}
Guessing letter: a
Server response: {'game_id': '39e3f404b4f5', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ _ _ t _ _ _ i _ e '}
Gues

Server response: {'game_id': 'f07f33acc75e', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ i _ s '}
Guessing letter: a
Server response: {'game_id': 'f07f33acc75e', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ a _ i _ s '}
Guessing letter: l
Server response: {'game_id': 'f07f33acc75e', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ a _ i _ s '}
Guessing letter: r
Server response: {'game_id': 'f07f33acc75e', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ r _ a r i _ s '}
Strat 1 returned no solution, changing to strat 2.
Strat 2 returned no solution, changing to strat 3.
Guessing letter: m
Server response: {'game_id': 'f07f33acc75e', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ r _ a r i _ s '}
Guessing letter: t
Server response: {'game_id': 'f07f33acc75e', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ _ r _ a r i _ s '}
Guessing letter: b
Server response: {'game_id': 'f07f33acc75e', 'status': 'ongoing', 'tries_remains': 1, 'w

Successfully start a new game! Game ID: 4dca6c3dbef0. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '4dca6c3dbef0', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ e _ '}
Guessing letter: r
Server response: {'game_id': '4dca6c3dbef0', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ e _ '}
Guessing letter: s
Server response: {'game_id': '4dca6c3dbef0', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ e _ '}
Guessing letter: l
Server response: {'game_id': '4dca6c3dbef0', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ e l '}
Guessing letter: n
Server response: {'game_id': '4dca6c3dbef0', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ e l '}
Guessing letter: c
Server response: {'game_id': '4dca6c3dbef0', 'status': 'ongoing', 'tries_remains': 3, 'word': 'c _ _ _ _ e l '}
Guessing letter: a
Server response: {'game_id': '4dca6c3dbef0', 'status': 'ongoing', 'tries_remains': 2, 'word': 

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

In [50]:
print(counter)
print(success)

38
[[0, 16, 11, 10, 24], [0, 5, 3, 15, 15]]


## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [51]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.391


In [52]:
total_recorded_runs

1000